In [1]:
import definitions as df
import data_manipulation as dm
import sql_def as sql
import os
import numpy as np
import matplotlib.pyplot as plt
import plots
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import math
import torch.nn.functional as F

c:\Users\alito\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Grabing the data from SQL dataset

In [ ]:
path_to_database = r"./data"
path_to_code = r"./"
os.chdir(path_to_database)
track_from_sql =  sql.data_from_sql("database.db", "SELECT * FROM TRACKMATEDATA")
os.chdir(path_to_code)
track_with_frame = df.separate_trajectories(track_from_sql)
trajectories1 = df.separate_data(track_with_frame, False)
trajectories = []
for temp in trajectories1:
    f2 = []
    c = 1
    for t1 in temp:
        f2.append(t1)
        c += 1
        if c == 10:
            break
    trajectories.append(f2)
trajectories

Making Fake data

In [2]:
total_size = 12000
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 20

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)

In [3]:
train_norm, test_norm = df.split_test_train(trajectories)
train, min_x, min_y, range_x, range_y = df.normalize_data(train_norm)
test, min_x, min_y, range_x, range_y = df.normalize_data(test_norm, min_x, min_y, range_x, range_y)
train_data, train_masked_point = df.mask_point_at_index(train, 6)
test_data, test_masked_point = df.mask_point_at_index(test, 6)

print(len(train_data))
print(len(test_masked_point))

10800
1200


In [4]:
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)

## DEBUG
print(src_data_tensor)
print("src_data_tensor shape:", src_data_tensor.shape)
print("tgt_data_tensor shape:", tgt_data_tensor.shape)
print("src_masks_tensor shape:", src_masks_tensor.shape)

hello
tensor([[[0.3367, 0.5637],
         [0.3382, 0.5637],
         [0.3396, 0.5637],
         ...,
         [0.3617, 0.5637],
         [0.3632, 0.5637],
         [0.3647, 0.5637]],

        [[0.8358, 0.1929],
         [0.8333, 0.1875],
         [0.8309, 0.1822],
         ...,
         [0.7935, 0.1015],
         [0.7910, 0.0961],
         [0.7885, 0.0908]],

        [[0.2601, 0.4213],
         [0.2653, 0.4217],
         [0.2704, 0.4221],
         ...,
         [0.3475, 0.4275],
         [0.3527, 0.4279],
         [0.3578, 0.4282]],

        ...,

        [[0.7968, 0.5370],
         [0.7971, 0.5385],
         [0.7973, 0.5400],
         ...,
         [0.8012, 0.5619],
         [0.8014, 0.5633],
         [0.8017, 0.5648]],

        [[0.6783, 0.6609],
         [0.6823, 0.6628],
         [0.6863, 0.6646],
         ...,
         [0.7463, 0.6929],
         [0.7503, 0.6947],
         [0.7543, 0.6966]],

        [[0.5914, 0.4391],
         [0.5972, 0.4405],
         [0.6029, 0.4418],
         

In [5]:
print(os.getcwd())
import model as md

input_dim = 2
hidden_dim = 64                                                                                   
output_dim = 2
rate = 0.0001

model = md.SelfSupervisedModel(in_channels=3, out_channels=128, input_dim=128, model_dim=256, num_heads=8, num_layers=6, output_dim=3)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate) ## w 


c:\Users\alito\Desktop\NanceLab\DeepLearning-Model\diff_transformers\project


c:\Users\alito\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
n_iters = 10

for epoch in range(n_iters):
    total_loss = 0
    for src, tgt, mask in dataloader:
        optimizer.zero_grad()
        output = model(src)
        masked_output = output # output[:, 6, :]
        loss = loss_function(masked_output, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{n_iters}, Loss: {total_loss / len(dataloader)}')

TypeError: forward() missing 3 required positional arguments: 'edge_index', 'src', and 'tgt'

In [8]:
## TEMP
n_iters = 40
learning_rate = 0.001
batch_size = 32
threshold = 0.01

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Example DataLoader setup

# Lists to store loss and accuracy values for each epoch
loss_values = []
accuracy_values = []

def compute_accuracy(predictions, targets, threshold=0.01):
    distances = torch.sqrt(torch.sum((predictions - targets) ** 2, dim=1))
    accurate_predictions = distances < threshold
    accuracy = torch.mean(accurate_predictions.float()) * 100
    return accuracy.item()

# Training loop
for epoch in range(n_iters):
    total_loss = 0
    model.train()  # Set the model to training mode
    for src, tgt, mask in dataloader:
        print(f'Src shape: {src.shape}, Tgt shape: {tgt.shape}, Mask shape: {mask.shape}')
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src, mask, src, tgt)
        
        # Compute loss
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    loss_values.append(avg_loss)
    print(f'Epoch {epoch+1}/{n_iters}, Loss: {avg_loss}')

    # Evaluation phase
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
        
        predictions = model(t_src_data_tensor, t_src_masks_tensor, t_src_data_tensor, t_tgt_data_tensor)
        
        accuracy = compute_accuracy(predictions, t_tgt_data_tensor, threshold=threshold)
        accuracy_values.append(accuracy)
        print(f'Accuracy after Epoch {epoch+1}/{n_iters}: {accuracy:.2f}%')

# Plotting loss and accuracy
epochs = range(1, n_iters + 1)

plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_values, 'b', label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy_values, 'g', label='Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Model Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

Src shape: torch.Size([32, 20, 2]), Tgt shape: torch.Size([32, 20, 2]), Mask shape: torch.Size([32, 20, 2])
tensor([[[False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [ True,  True],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False]],

        [[False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [ True,  True],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False, False],
         [False

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 32 but got size 2 for tensor number 1 in the list.

In [ ]:
model.eval()
t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
X_test = t_src_data_tensor[3]
with torch.no_grad():
    predictions = model(X_test)
    print(X_test)
    print("Predictions:", predictions)

In [ ]:
t_src_data_tensor[3]

In [ ]:
def compute_accuracy(predictions, targets, threshold=0.000001):
    distances = torch.sqrt(torch.sum((predictions - targets) ** 2, dim=1))
    accurate_predictions = distances < threshold
    accuracy = torch.mean(accurate_predictions.float()) * 100
    return accuracy.item()
predictions = model(t_src_data_tensor)
acc = compute_accuracy(predictions, t_tgt_data_tensor, 0.05)
print('accuracy of model to predict a point: {:.2f} %'.format(acc))

In [ ]:
## TEMP

loss_values = []
accuracy_values = []

for epoch in range(40):
    total_loss = 0
    model.train()  # Set the model to training mode
    for src, tgt, mask in dataloader:
        optimizer.zero_grad()
        output = model(src)
        masked_output = output  # output[:, 6, :]
        loss = loss_function(masked_output, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    loss_values.append(avg_loss)
    print(f'Epoch {epoch+1}/{n_iters}, Loss: {avg_loss}')

    # Evaluation phase
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
        print(t_tgt_data_tensor[2])
        predictions = model(t_src_data_tensor)
        print(predictions[2])
        accuracy = compute_accuracy(predictions, t_tgt_data_tensor, threshold=0.01)
        print(accuracy)
        accuracy_values.append(accuracy)
        print(f'Accuracy after Epoch {epoch+1}/{n_iters}: {accuracy:.2f}%')

# Plotting loss and accuracy
epochs = range(1, 40 + 1)

plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_values, 'b', label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy_values, 'r', label='Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Model Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
predictions = model(t_src_data_tensor)
print(t_src_data_tensor)
# print(predictions)

In [ ]:
def unnormalize_data(normalized_data, min_x, min_y, range_x, range_y):
    unnormalized_data = [[(float(point[0] * range_x + min_x), float(point[1] * range_y + min_y)) if point is not None else None for point in seq] for seq in normalized_data]
    return unnormalized_data

def unnormalize_data_pred(normalized_data, min_x, min_y, range_x, range_y):
    unnormalized_data = [(float(seq[0] * range_x + min_x), float(seq[1] * range_y + min_y)) for seq in normalized_data]
    return unnormalized_data

In [ ]:
data = unnormalize_data(t_src_data_tensor, min_x, min_y, range_x, range_y)
pred = unnormalize_data_pred(predictions, min_x, min_y, range_x, range_y)
data = torch.FloatTensor(data)
pred = torch.FloatTensor(pred)

In [ ]:
t_src_data_tensor = torch.tensor(t_src_data_tensor)
predictions = torch.tensor(predictions)

# Create a plot for tensors
plt.figure(figsize=(8, 6))
plt.scatter(data[:, 0], data[:, 1], color='blue', label='Tensor Data Points')
plt.scatter(pred[0], pred[1], color='red', label='Prediction')
plt.title('Tensor Data and Prediction')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.xlim(0, 2000)
plt.ylim(0, 2000)
plt.legend()
plt.grid(True)
plt.savefig("straightLine.png")

# Actual Model


In [ ]:
src_data_tensor[:, 0]

In [ ]:
t_src_data_tensor = torch.tensor(t_src_data_tensor)
predictions = torch.tensor(predictions)

# Create a plot for tensors
plt.figure(figsize=(8, 6))
plt.scatter(data[:, 0], data[:, 1], color='blue', label='Tensor Data Points')
plt.scatter(pred[0], pred[1], color='red', label='Prediction')
plt.title('Tensor Data and Prediction')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.xlim(0, 2000)
plt.ylim(0, 2000)
plt.legend()
plt.grid(True)
plt.savefig("straightLine.png")

In [ ]:
def plot_syn(random_trajectories=[], pred=[]):
    if(len(random_trajectories) == 0):
        random_trajectories = dm.create_synthetic(3)
    for trajectory in random_trajectories:
        print(trajectory)
        x_values = [point[0] for point in trajectory]
        y_values = [point[1] for point in trajectory] 
        plt.plot(x_values, y_values)
        print("hi")
    plt.scatter(pred[0], pred[1], color='red', label='Prediction')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(0, 2000)
    plt.ylim(0, 2000)
    plt.title('Trajectories')
    plt.grid(True)
    plt.show()

In [ ]:
pred[4][0]

In [ ]:
first_ten_arrays = data[3:4]

# Now exclude the element at index 6 from each of these sub-arrays
result_tensor = torch.cat((first_ten_arrays[:, :6], first_ten_arrays[:, 7:]), dim=1)

plot_syn(result_tensor, pred[4])

In [ ]:
data[3:4]

In [ ]:
src_data_tensor[:10]

In [ ]:
def compute_accuracy(predictions, targets, threshold=0.1):
    distances = torch.sqrt(torch.sum((predictions - targets) ** 2, dim=1))
    accurate_predictions = distances < threshold
    accuracy = torch.mean(accurate_predictions.float()) * 100
    return accuracy.item()
predictions = model(t_src_data_tensor)
acc = compute_accuracy(predictions, t_tgt_data_tensor, 0.05)
print('accuracy of model to predict a point: {:.2f} %'.format(acc))

In [ ]:
t_tgt_data_tensor

In [ ]:
data = unnormalize_data_pred(t_tgt_data_tensor, min_x, min_y, range_x, range_y)
torch.tensor(data)

In [ ]:
data = unnormalize_data_pred(t_tgt_data_tensor, min_x, min_y, range_x, range_y)
pred = unnormalize_data_pred(predictions, min_x, min_y, range_x, range_y)
acc = compute_accuracy(torch.tensor(pred), torch.tensor(data), 10)
print('accuracy of model to predict a point: {:.2f} %'.format(acc))

In [ ]:
torch.tensor(x[:1])

In [ ]:
def plot_syn(random_trajectories=[], pred=[], y=[]):
    if(len(random_trajectories) == 0):
        random_trajectories = dm.create_synthetic(3)
    for trajectory in random_trajectories:
        print(trajectory)
        x_values = [point[0] for point in trajectory]
        y_values = [point[1] for point in trajectory] 
        plt.plot(x_values, y_values)
        print("hi")
    plt.scatter(pred[0], pred[1], color='red', label='Prediction')
    plt.scatter(y[0], y[1], color='blue', label='Actual') 

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(650, 700)
    plt.ylim(650, 700)
    plt.title('Trajectories')
    plt.grid(True)
    plt.legend()
    plt.savefig("straighLine.png")
    plt.show()

In [ ]:
single_point = torch.tensor(y[3]).unsqueeze(0)  # Make it (1, 2)
# Expand this point to match the first dimension of first_ten_arrays
expanded_point = single_point.expand(first_ten_arrays.size(0), -1, -1)  # Expand to (10, 1, 2)

# Concatenate the point between the two segments
result_tensor = torch.cat((first_ten_arrays[:, :6], expanded_point, first_ten_arrays[:, 7:]), dim=1)

print(result_tensor)

In [ ]:
y[3]

In [ ]:
for i in range(200):
  if(np.abs(y[i][0] - y[i][1]) < 100):
    print(i)
    print(y[i])

In [ ]:
x = unnormalize_data(t_src_data_tensor, min_x, min_y, range_x, range_y)
y = unnormalize_data_pred(t_tgt_data_tensor, min_x, min_y, range_x, range_y)

index = 22

print(x[index])
print(pred[index])
print(y[index])
first_ten_arrays = torch.tensor(x[index:index+1])

# Now exclude the element at index 6 from each of these sub-arrays
single_point = torch.tensor(y[index]).unsqueeze(0)  # Make it (1, 2)
# Expand this point to match the first dimension of first_ten_arrays
expanded_point = single_point.expand(first_ten_arrays.size(0), -1, -1)  # Expand to (10, 1, 2)

# Concatenate the point between the two segments
result_tensor = torch.cat((first_ten_arrays[:, :6], expanded_point, first_ten_arrays[:, 7:]), dim=1)
              
plot_syn(result_tensor, torch.tensor(pred[index]),  torch.tensor(y[index]))

In [ ]:
torch.tensor(x[:1])

In [ ]:
print(t_tgt_data_tensor[0])
print(predictions[0])

In [ ]:
(0 * range_x + min_x)

In [ ]:
def normalize_data(data, min_x=0, min_y=0, range_x=0, range_y=0):
    # (x, y) -> needs to be the data, otherwise i gotta rewrite possibly
    # can assume with none
    all_points = [point for seq in data for point in seq if point is not None]
    if range_x or range_y == 0 or min_x == 0 or min_y == 0:
        all_x = [point[0] for point in all_points]
        all_y = [point[1] for point in all_points] 

        min_x = min(all_x)
        max_x = max(all_x)
        min_y = min(all_y)
        max_y = max(all_y)
        range_x = max_x - min_x
        range_y = max_y - min_y
        print(min_x)

    # Normalize data
    normalized_data = []
    for seq in data:
        normalized_seq = [(float(point[0] - min_x) / range_x, float(point[1] - min_y) / range_y) if point is not None else None for point in seq]
        normalized_data.append(normalized_seq)

    return normalized_data, min_x, min_y, range_x, range_y

train, min_x, min_y, range_x, range_y = normalize_data(train_norm)

In [ ]:
all_points = [point for seq in train_norm for point in seq if point is not None]

all_x = [point[0] for point in all_points]
all_y = [point[1] for point in all_points] 

print(min(all_x))
m = min(all_y)
for ele in all_points:
  if ele[1] == m:
    print(ele)
# print(len(train_norm))

In [ ]:

    # Example DataLoader setup

    # Lists to store loss and accuracy values for each epoch
    # loss_values = []
    # accuracy_values = []

    # # Training loop
    # for epoch in range(n_iters):
    #     total_loss = 0
    #     model.train()  # Set the model to training mode
    #     for src, tgt, mask in dataloader:
    #         print(f'Src shape: {src.shape}, Tgt shape: {tgt.shape}, Mask shape: {mask.shape}')
    #         optimizer.zero_grad()
            
    #         # Forward pass
    #         output = model(src, mask, src, tgt)
            
    #         # Compute loss
    #         loss = criterion(output, tgt)
    #         loss.backward()
    #         optimizer.step()

    #         total_loss += loss.item()
    #     avg_loss = total_loss / len(dataloader)
    #     loss_values.append(avg_loss)
    #     print(f'Epoch {epoch+1}/{n_iters}, Loss: {avg_loss}')

    #     # Evaluation phase
    #     model.eval()  # Set the model to evaluation mode
    #     with torch.no_grad():
    #         t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
            
    #         predictions = model(t_src_data_tensor, t_src_masks_tensor, t_src_data_tensor, t_tgt_data_tensor)
            
    #         accuracy = compute_accuracy(predictions, t_tgt_data_tensor, threshold=threshold)
    #         accuracy_values.append(accuracy)
    #         print(f'Accuracy after Epoch {epoch+1}/{n_iters}: {accuracy:.2f}%')

    # # Plotting loss and accuracy
    # epochs = range(1, n_iters + 1)

    # plt.figure(figsize=(12, 5))

    # # Plot loss
    # plt.subplot(1, 2, 1)
    # plt.plot(epochs, loss_values, 'b', label='Loss')
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.title('Training Loss')
    # plt.legend()

    # # Plot accuracy
    # plt.subplot(1, 2, 2)
    # plt.plot(epochs, accuracy_values, 'g', label='Accuracy')
    # plt.xlabel('Epochs')
    # plt.ylabel('Accuracy (%)')
    # plt.title('Model Accuracy')
    # plt.legend()

    # plt.tight_layout()
    # plt.show()